In [1]:
%pip install requests
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [20]:
from dotenv import load_dotenv
import requests
import os
load_dotenv() 
API_KEY = os.getenv("UMLS")


In [33]:
def get_tgt(API_KEY):
    url = "https://utslogin.nlm.nih.gov/cas/v1/api-key"
    params = {'apikey': API_KEY}
    response = requests.post(url, data=params)
    tgt_url = response.headers['location']
    return tgt_url

In [34]:
def get_st(tgt):
    response = requests.post(tgt, data={'service': 'http://umlsks.nlm.nih.gov'})
    return response.text

In [36]:
def search_umls(symptom, st):
    base_url = "https://uts-ws.nlm.nih.gov/rest/search/current"
    params = {
        "string": symptom,
        "ticket": st
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    results = data['result']['results']
    if results:
        return results[0]['ui']  # The CUI
    return None

In [37]:
def get_related_diseases(cui, st):
    url = f"https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{cui}/relations"
    params = {"ticket": st}
    response = requests.get(url, params=params)
    data = response.json()
    diseases = []
    for rel in data.get('result', []):
        if "may_be_diagnosed_by" in rel.get("relationLabel", "").lower() or \
           "associated_with" in rel.get("relationLabel", "").lower():
            diseases.append(rel['relatedIdName'])
    return diseases

In [40]:
def search_symptoms_from_umls(st, page=1):
    url = "https://uts-ws.nlm.nih.gov/rest/search/current"
    params = {
        "ticket": st,
        "pageNumber": page,
        "string": "headache",  # Use a generic seed term
        "returnIdType": "concept"
    }

    response = requests.get(url, params=params)
    print(f"🔎 UMLS Search Response Status: {response.status_code}")

    try:
        data = response.json()
        print("📥 Raw response (truncated):", str(data)[:200])  # Print first 200 chars
        results = data["result"]["results"]
        return [r["name"] for r in results if r["ui"] != "NONE"]
    except KeyError:
        print("result' not in response. Full data:", data)
        return []

In [10]:
%pip install numpy==1.26.4 --force-reinstall --no-cache-dir

   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
    --------------------------------------- 0.3/15.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.5 MB 1.7 MB/s eta 0:00:09
   -- ------------------------------------- 1.0/15.5 MB 1.9 MB/s eta 0:00:08
   ---- ----------------------------------- 1.6/15.5 MB 2.2 MB/s eta 0:00:07
   ------ --------------------------------- 2.4/15.5 MB 2.5 MB/s eta 0:00:06
   -------- ------------------------------- 3.1/15.5 MB 2.7 MB/s eta 0:00:05
   ---------- ----------------------------- 3.9/15.5 MB 2.9 MB/s eta 0:00:04
   ------------ --------------------------- 5.0/15.5 MB 3.1 MB/s eta 0:00:04
   -------------- ------------------------- 5.8/15.5 MB 3.2 MB/s eta 0:00:04
   ----------------- ---------------------- 6.8/15.5 MB 3.4 MB/s eta 0:00:03
   -------------------- ------------------- 7.9/15.5 MB 3.6 MB/s eta 0:00:03
   ------------------------ --------------- 9.4/15.5 MB 3.9 MB/s eta 0:00:02
   ----------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.2.1 which is incompatible.


In [8]:
import numpy
print(numpy.__version__)  

2.1.3


In [10]:
%pip uninstall pandas -y
%pip install pandas

Found existing installation: pandas 2.3.0
Uninstalling pandas-2.3.0:
  Successfully uninstalled pandas-2.3.0
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.
You can safely remove it manually.


  Using cached pandas-2.3.0-cp312-cp312-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.3.0-cp312-cp312-win_amd64.whl (11.0 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.2.1 which is incompatible.


In [9]:
import pandas as pd

In [41]:
tgt = get_tgt(API_KEY)
st = get_st(tgt)

all_symptoms = []
for page in range(1, 4):  # You can increase pages if needed
    print(f"Fetching symptoms page {page}...")
    all_symptoms.extend(search_symptoms_from_umls(st, page))


all_symptoms = list(set(all_symptoms))
print(f"Total unique symptoms fetched: {len(all_symptoms)}")
results = []

for symptom_str in all_symptoms:
    print(f"\nProcessing symptom: {symptom_str}")
    st = get_st(tgt)
    print(f"Service Ticket (ST): {st}")
    cui = search_umls(symptom_str, st)
    print(f"CUI: {cui}")

    if cui and cui != "NONE":
        diseases = get_related_diseases(cui, st)
        print(f"🩺 Related diseases: {diseases}")
        results.append({
            "symptoms": symptom_str,
            "diseases": "; ".join(diseases) if diseases else "Not Found"
        })
    else:
        print("No valid CUI found.")
        results.append({
            "symptoms": symptom_str,
            "diseases": "Not Found"
        })


# STEP 4: Save results to CSV
pd.DataFrame(results).to_csv("symptom_disease.csv", index=False)

print("\n symptom_disease.csv created successfully.")

Fetching symptoms page 1...
🔎 UMLS Search Response Status: 200
📥 Raw response (truncated): {'pageSize': 25, 'pageNumber': 1, 'result': {'classType': 'searchResults', 'results': [{'ui': 'C0018681', 'rootSource': 'MTH', 'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2025AA/CUI/C0018681', 'na
Fetching symptoms page 2...
🔎 UMLS Search Response Status: 401
📥 Raw response (truncated): {'name': 'UnauthorizedError', 'status': 401, 'message': '401 - Invalid ST - Documentation: https://documentation.uts.nlm.nih.gov/rest/authentication.html'}
result' not in response. Full data: {'name': 'UnauthorizedError', 'status': 401, 'message': '401 - Invalid ST - Documentation: https://documentation.uts.nlm.nih.gov/rest/authentication.html'}
Fetching symptoms page 3...
🔎 UMLS Search Response Status: 401
📥 Raw response (truncated): {'name': 'UnauthorizedError', 'status': 401, 'message': '401 - Invalid ST - Documentation: https://documentation.uts.nlm.nih.gov/rest/authentication.html'}
result' not in respons